In [ ]:
#!/usr/bin/python
'''----------------Import modules START------------------'''

import sys
import time
import re

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk import FreqDist
nltk.download('brown')
from nltk.corpus import brown

from ipypb import irange

from operator import itemgetter, attrgetter
from collections import deque
from sortedcontainers import SortedSet,SortedList

'''----------------Import modules END--------------------'''

'''----------------myDict Class START--------------------'''

class myDict:
    # INITIALIZE
    def __init__:
        self.previousWord = ""
        self.root = DawgNode()
        self.uncheckedNodes = []
        self.minimizedNodes = {}
    
    # INSERT WORD
    def insert( self, word ):
        if word < self.previousWord:
            raise Exception("Error: Words must be inserted in alphabetical order.")
        
        commonPrefix = 0
        maxPrefix =  min( len(word),len(self.previousWord) )
        for i in range( maxPrefix ):
            if word[i] != self.previousWord[i]: break
            commonPrefix += 1
        self._minimize( commonPrefix )
            
            
'''----------------myDict Class END----------------------'''

DICTIONARY = SortedSet([])
QUERY = sys.argv[1:]

frequency_list = FreqDist(w.lower().strip("-") for w in brown.words() if (re.search('[a-zA-Z]+',w) and lookUp_update(w)) )

ACCEPTED = SortedSet([])
CUSTOMDICT = SortedSet([])
ALLWORDS = SortedSet([])
'''----------------Loading the dictionary START------------------'''
def read_custom_dict():
    with open("./hardcode/custom_dict.txt",'r') as file:
        for line in file:
            word = "".join(line.split())
            CUSTOMDICT.add(word.lower())
'''----------------Loading the dictionary END--------------------'''
def lookUp_update(word):
    if word in CUSTOMDICT or wn.synsets(word,'asrnv') or word in ACCEPTED:
        acceptWord(word)
        return True
    return False

def lookUp_update(word):
    if word in CUSTOMDICT or wn.synsets(word,'asrnv') or word in ACCEPTED:
        acceptWord(word)
        return True
    return False

def acceptWord(word):
    ACCEPTED.add(word)
    notACCEPTED.discard(word)

def rejectWord(word):
    ACCEPTED.discard(word)
    notACCEPTED.add(word)



In [ ]:
pattern = re.compile(r"([\w\-\']*[a-zA-Z]+[\w\-\']*)")
with open("mobydick.txt") as file:
    for line in file:                      # foreach line
        for match in re.finditer(pattern, line):
            word = line[match.start():match.end()].lower()
            lookUp_update(word)
                
                
# This class represents a node in the directed acyclic word graph (DAWG). It
# has a list of edges to other nodes. It has functions for testing whether it
# is equivalent to another node. Nodes are equivalent if they have identical
# edges, and each identical edge leads to identical states. The __hash__ and
# __eq__ functions allow it to be used as a key in a python dictionary.
class DawgNode:
    NextId = 0
    
    def __init__(self):
        self.id = DawgNode.NextId
        DawgNode.NextId += 1
        self.final = False
        self.edges = {}

    def __str__(self):        
        arr = []
        if self.final: 
            arr.append("1")
        else:
            arr.append("0")

        for (label, node) in self.edges.items():
            arr.append( label )
            arr.append( str( node.id ) )

        return "_".join(arr)

    def __hash__(self):
        return self.__str__().__hash__()

    def __eq__(self, other):
        return self.__str__() == other.__str__()

class Dawg:
    def __init__(self):
        self.previousWord = ""
        self.root = DawgNode()

        # Here is a list of nodes that have not been checked for duplication.
        self.uncheckedNodes = []

        # Here is a list of unique nodes that have been checked for
        # duplication.
        self.minimizedNodes = {}

    def insert( self, word ):
        if word < self.previousWord:
            raise Exception("Error: Words must be inserted in alphabetical " +
                "order.")

        # find common prefix between word and previous word
        commonPrefix = 0
        for i in range( min( len( word ), len( self.previousWord ) ) ):
            if word[i] != self.previousWord[i]: break
            commonPrefix += 1

        # Check the uncheckedNodes for redundant nodes, proceeding from last
        # one down to the common prefix size. Then truncate the list at that
        # point.
        self._minimize( commonPrefix )

        # add the suffix, starting from the correct node mid-way through the
        # graph
        if len(self.uncheckedNodes) == 0:
            node = self.root
        else:
            node = self.uncheckedNodes[-1][2]

        for letter in word[commonPrefix:]:
            nextNode = DawgNode()
            node.edges[letter] = nextNode
            self.uncheckedNodes.append( (node, letter, nextNode) )
            node = nextNode

        node.final = True
        self.previousWord = word

    def finish( self ):
        # minimize all uncheckedNodes
        self._minimize( 0 );

    def _minimize( self, downTo ):
        # proceed from the leaf up to a certain point
        for i in range( len(self.uncheckedNodes) - 1, downTo - 1, -1 ):
            (parent, letter, child) = self.uncheckedNodes[i];
            if child in self.minimizedNodes:
                # replace the child with the previously encountered one
                parent.edges[letter] = self.minimizedNodes[child]
            else:
                # add the state to the minimized nodes.
                self.minimizedNodes[child] = child;
            self.uncheckedNodes.pop()

    def lookup( self, word ):
        node = self.root
        for letter in word:
            if letter not in node.edges: return False
            node = node.edges[letter]

        return node.final

    def nodeCount( self ):
        return len(self.minimizedNodes)

    def edgeCount( self ):
        count = 0
        for node in self.minimizedNodes:
            count += len(node.edges)
        return count

        
dawg = Dawg()
WordCount = 0
words = open(DICTIONARY, "rt").read().split()
words.sort()
start = time.time()    
for word in words:
    WordCount += 1
    dawg.insert(word)
    if ( WordCount % 100 ) == 0: print(WordCount)
dawg.finish()
print("Dawg creation took ",(time.time()-start) )

EdgeCount = dawg.edgeCount()
print( "Read %d words into %d nodes and %d edges" % ( WordCount,
        dawg.nodeCount(), EdgeCount ))
print( "This could be stored in as little as %d bytes" % (EdgeCount * 4)    )

for word in QUERY:
    if not dawg.lookup( word ):
        print( "%s not in dictionary." % word)
    else:
        print("%s is in the dictionary." % word)